In [1]:
import subprocess
import re
from pathlib import Path
import time

In [2]:
# === Configuration ===
data_path = "/xustrg0/2024B8049"
run_number = "222753"
process_path = "/UserData/fperakis/test_data_2025/"
mask_path = process_path+"utilities/empty_mask.npy"
output_path = process_path+"processed"
poni_file = process_path+"utilities/geometry_test.poni"
nbins = 150
n_phi = 36
n_chunks = 10  # total number of chunks

In [3]:
subprocess.run([
    "python", "process.py",
    "--data_path", data_path,
    "--run_number", run_number,
    "--mask_path", mask_path,
    "--output_path", output_path,
    "--poni_file", poni_file,
    "--nbins", str(nbins),
    "--n_phi", str(n_phi),
    "--n_chunks", str(n_chunks)
])


Array job submitted with ID: 8423233
Merge job submitted successfully with job ID: 8423234.fep01


CompletedProcess(args=['python', 'process.py', '--data_path', '/xustrg0/2024B8049', '--run_number', '222753', '--mask_path', '/UserData/fperakis/test_data_2025/utilities/empty_mask.npy', '--output_path', '/UserData/fperakis/test_data_2025/processed', '--poni_file', '/UserData/fperakis/test_data_2025/utilities/geometry_test.poni', '--nbins', '150', '--n_phi', '36', '--n_chunks', '10'], returncode=0)

In [13]:
# === Submit array job ===
env_vars = ",".join([
    f"DATA_PATH={data_path}",
    f"RUN_NUMBER={run_number}",
    f"MASK_PATH={mask_path}",
    f"OUTPUT_PATH={output_path}",
    f"PONI_FILE={poni_file}",
    f"NBINS={nbins}",
    f"N_PHI={n_phi}",
    f"N_CHUNKS={n_chunks}"
])

submit_cmd = [
    "qsub", "-J", f"0-{n_chunks-1}",
    "-v", env_vars,
    "submit_chunk.pbs"
]

array_result = subprocess.run(submit_cmd, capture_output=True, text=True)

# === Extract array job ID ===
match = re.search(r"(\d+)", array_result.stdout)
if not match:
    raise RuntimeError("Failed to parse array job ID. Output:\n" + array_result.stdout)

array_job_id = match.group(1)
print(f"\nArray job submitted with ID: {array_job_id}")

# === Submit dependent merge job ===

# Define parameters
output_file = Path(output_path) / f"Iq_{run_number}_combined.h5"

job_id_base = array_job_id+"[]" # quick fix, check for inconsistencies

# Submit the PBS job
merge_cmd = [
    "qsub",
    "-W", f"depend=afterok:{job_id_base}",
    "-v", f"INPUT_DIR={Path(output_path)},RUN_NUMBER={run_number},OUTPUT_FILE={output_file}",
    "combine_chunks.pbs"
]

result = subprocess.run(merge_cmd, capture_output=True, text=True)

# Output results
if result.returncode == 0:
    merge_job_id = result.stdout.strip()
    print(f"Merge job submitted successfully with job ID: {merge_job_id}")
else:
    print("Failed to submit merge job.")
    print(result.stderr)


Array job submitted with ID: 8422036
Merge job submitted successfully with job ID: 8422037.fep01


In [4]:
# === Check job status ===
!qstat -u fperakis


fep01: 
                                                            Req'd  Req'd   Elap
Job ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time
--------------- -------- -------- ---------- ------ --- --- ------ ----- - -----
8423233[].fep01 fperakis serial   iq_chunk      --    1   1    4gb 00:30 B   -- 
8423234.fep01   fperakis serial   combine_c*    --    1   1    4gb 00:10 H   -- 


In [16]:
!ls -lh /UserData/fperakis/test_data_2025/processed/

total 248M
-rw-r--r-- 1 fperakis nilsson 248M May 15 23:03 Iq_222753_combined.h5
